#Please run this .ipynb in Google Colab with an associated Google Drive account. You may want to consult the following two links to get more information about how to get FREE access to these resources.
https://www.google.com/drive/

https://colab.research.google.com/

##(i) Here we import all necessary libraries
Please put all your files at a desired location. For example, under "/content/drive/MyDrive/Colab Notbooks/USC_BA_estimator/" folder and load our packages. 



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

sys.path.append("/home/usuaris/imatge/joan.manel.cardenas/brainAge/USC_BA_estimator/function")

#import nibabel.freesurfer.mghformat as mgh
from scipy.ndimage import zoom
import datetime
import pandas as pd
from preprocess import processmgz
from load_female_data import female_data
import savepath_updated
import repadding
from pandas import read_csv
import tensorflow as tf
from tensorflow import keras
import modelcnn
import saliencymaps
import multiprocessing as mp

2024-03-04 17:03:29.621125: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 17:03:29.621201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 17:03:29.623441: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 17:03:29.636092: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 17:03:32.796008: W tensorflow/compiler/tf2

##(ii) Main function.


In [2]:
def mainfunction(dir_test_brain_age, path_to_model_weights):
    # we supposed the brain.mgz files are saved under this format: "/path_to_brain_mri/subject_id/brain.mgz". 
    #path_of_mgz = os.listdir(path_to_brain_mri)
    female_info_list = []
    subj_id = []
    brains_tmp = []
    female_info_list = female_data()
    # Obtener los IDs de los sujetos
    subj_id = [file_info[0] for file_info in female_info_list]

    # Obtener los datos de las imágenes MRI
    brains_tmp = [file_info[1] for file_info in female_info_list]

    #for mgz_id in path_of_mgz:
      #if os.path.isdir(path_to_brain_mri+"/"+mgz_id) and os.path.exists(path_to_brain_mri+"/"+mgz_id+"/brain.mgz"):
        #subj_id.append(mgz_id)
        #brains_tmp.append(zoom(mgh.load(path_to_brain_mri+"/"+mgz_id+"/brain.mgz").get_data(), (0.5, 0.5, 0.5)))
    
    brains, coordinates = processmgz(brains_tmp)
    (
        brain_save_path,
        coordinates_save_path,
        coordinates_save_path_csv,
    ) = savepath_updated.filename_brainnpy(dir_test_brain_age)

    # save the cropped brains
    np.save(brain_save_path, brains)

    # save information for repadding.
    np.save(coordinates_save_path, coordinates)
    df_coordinates = pd.DataFrame(coordinates)
    df_coordinates.columns = ["x", "y", "z"]
    df_coordinates["subj_id"] = subj_id
    df_coordinates.to_csv(coordinates_save_path_csv)

    # Build model. 
    model = modelcnn.get_model(width=82, height=86, depth=100)

    # Load best weights.
    model.load_weights(path_to_model_weights)

    # make brain age predictions
    predictions = model.predict(brains) + 22

    # save the prediction results. Of note, the input subject ids should be saved as csv.
    
    BA_save_path_csv = savepath_updated.filename_pred(dir_test_brain_age)
    df_BA = pd.DataFrame(predictions)
    df_BA.columns = ["BA"]
    df_BA["subj_id"] = subj_id
    df_BA.to_csv(BA_save_path_csv)

    # Generate saliency mapes
    saliency_map_list = saliencymaps.smap(model, brains)

    # Repadding the saliency maps
    coordinates = np.load(coordinates_save_path)
    repadded_saliency_maps = repadding.repadding(saliency_map_list, coordinates)

    # Save the saliency maps
    smap_path = savepath_updated.filename_smap(dir_test_brain_age)
    np.save(smap_path, repadded_saliency_maps)
    return df_BA, repadded_saliency_maps


##(iii) Execute main function. 
"dir_test_brain_age" is the path to the folder of all the functions. \\
"path_to_model_weights" is the path to the h5 file. \\
"path_to_brain_mri" is the path to all the brain.mgz files. Here, we supposed the brain.mgz files are saved under this format: "/path_to_brain_mri/subject_id/brain.mgz". \\

The predicted BA will be saved as csv files under "dir_test_brain_age". \\
The csv file is saved in this format "BA-year-month-day-hour:min:sec.csv" \\

The processed saliency maps will be saved under "dir_test_brain_age."\\
The saliency maps will be saved in this format "Saliency_maps-year-month-day-hour:min:sec.csv" \\




In [3]:
# path to the folder
dir_test_brain_age = "/home/usuaris/imatge/joan.manel.cardenas/brainAge"

# path to the h5 file
path_to_model_weights = os.path.join(
    dir_test_brain_age, "USC_BA_estimator/function/model/saved-model-317.h5"
)

# path to the brain.mgz files

#path_to_brain_mri = os.path.join(
#    dir_test_brain_age, "brain_mri"
#)

# the mainfunction will automatically save and output the predicted BAs and saliency maps.
predicted_BA, repadded_saliency_maps = mainfunction(
    dir_test_brain_age, path_to_model_weights
)


Se encontró el archivo 'T1_brain_to_MNI.nii.gz' dentro del archivo .zip para el sujeto con ID 1000905.
Se encontró el archivo 'T1_brain_to_MNI.nii.gz' dentro del archivo .zip para el sujeto con ID 1002257.
Se encontró el archivo 'T1_brain_to_MNI.nii.gz' dentro del archivo .zip para el sujeto con ID 1002839.
Dimensiones de brains antes de convertirlo en un array de NumPy: 3 x (128, 128, 128)
Dimensiones de X: (3, 128, 128, 128)


2024-03-04 17:03:52.580012: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


1/1 [==============================] - 1s 1s/step


In [4]:
predicted_BA

,BA,subj_id
0,88.947800,1000905
1,58.767845,1002257
2,145.199203,1002839
